In [1]:
from iteru import *

In [2]:
aoi = GERD_aoi_dam

In [3]:
# Map.add_layer_widgets(ee.FeatureCollection(aoi),{'color':'blue'},'AOI')

In [363]:
SAR = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.equals('relativeOrbitNumber_start', 50))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('resolution_meters',10))\
    .filterBounds(aoi)\
    .filter(ee.Filter.eq('system:time_start',1627442724000))\
    .select(['VV','VH'])\
    .median()

In [364]:
# VV_smooth = SAR.select('VV').focal_median(50,'circle','meters').rename('VV_Filtered')

In [365]:
VV_smooth = ee.Image(toDB(RefinedLee(toNatural(SAR.select(['VV']))))).rename('VV_Filtered')

In [366]:
histogram = VV_smooth.select('VV_Filtered').reduceRegion(               
            reducer = ee.Reducer.histogram().combine('mean', None, True).combine('variance', None, True),
            geometry = aoi, 
            scale = 10,
            bestEffort = True   )

In [367]:
otsu_threshold = otsu(histogram.get('VV_Filtered_histogram'))

In [ ]:
otsu_threshold.getInfo()

In [ ]:
water_mask = VV_smooth.select('VV_Filtered').lt(ee.Number(otsu_threshold).subtract(ee.Number(3.88)).getInfo()).selfMask().rename('water_mask')

In [ ]:
ee.Number(otsu_threshold).subtract(ee.Number(3.88)).getInfo()

In [ ]:
water_mask = water_mask.select('water_mask').clip(aoi)

In [ ]:
# Map.add_ee_layer(water_mask,{'palette': ['00FFFF', '0000FF'], 'bands':['water_mask']},'SARF')

In [ ]:
feature = ee.Image(1).updateMask(water_mask).reduceToVectors(
           geometry = water_mask.geometry(),
           crs ='EPSG:32636',
           scale = 10 , 
           geometryType = 'polygon',
           eightConnected = False,
           labelProperty =  'water_cover',
           bestEffort = True
          )

In [ ]:
def calc_area(feature):
    
    area = feature.geometry().area(maxError = 1)
    
    return feature.set({'Area':area})

In [ ]:
feature = feature.map(calc_area)

In [ ]:
lake_feature = feature.sort('Area', False).first()

In [ ]:
# Map.add_layer_widgets(ee.FeatureCollection(lake_feature.geometry()),{'color':'blue'},'Reservoir')

In [ ]:
lake_area = lake_feature.geometry().area(maxError = 1) 

In [ ]:
lake_area.getInfo()/1e6

In [ ]:
lake_dem = elevation_dataset.clip(lake_feature)

In [ ]:
max_ele = lake_dem.reduceRegion(
                         reducer = ee.Reducer.max(),
                         geometry = lake_dem.geometry(), 
                         scale = 30, 
                         crs = 'EPSG:32636',
                        maxPixels = 1e11).get('DSM')

In [ ]:
water_level = max_ele.getInfo()

In [ ]:
water_level

In [ ]:
elevations = lake_dem.reduceRegion(
        reducer=ee.Reducer.toList(), 
        geometry=lake_dem.geometry(),
        maxPixels=1e11,
        scale=30,
        crs = 'EPSG:32636',
        bestEffort = True
    ).get('DSM')

In [ ]:
elev_pixles_num = ee.List(elevations).length()

In [ ]:
elev_sum = ee.List(elevations).reduce(ee.Reducer.sum())

In [ ]:
((water_level*elev_pixles_num.getInfo()-elev_sum.getInfo())*900)/(1e9)

In [ ]:
test = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.equals('relativeOrbitNumber_start', 50))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('resolution_meters',10))\
    .filterBounds(aoi)\
    .filterDate( '2021-07-28','2021-07-29')\
    .select(['VV','VH'])\
    

In [ ]:
get_imgCol_dates_dict(test)

In [ ]:
dates = ['2020-06-03',
         '2020-06-15',
         '2020-06-27',
         '2020-07-09',
         '2020-07-21',
         '2020-08-02',
         '2020-08-14',
         '2020-08-26',
         '2020-09-07',
         '2020-09-19',
         '2020-10-01',
         '2020-10-13',
         '2020-10-25',
         '2020-11-06',
         '2020-11-18',
         '2020-11-30',
         '2020-12-12',
         '2021-01-05',
         '2021-01-17',
         '2021-01-29',
         '2021-02-10',
         '2021-02-22',
         '2021-03-06',
         '2021-03-18',
         '2021-03-30',
         '2021-04-11',
         '2021-04-23',
         '2021-05-05',
         '2021-05-17',
         '2021-05-29',
         '2021-06-10',
         '2021-06-22',
         '2021-07-04',
         '2021-07-16',
         '2021-07-28',
         '2021-08-09',
         '2021-08-21',
         '2021-09-02',
         '2021-09-14',
         '2021-09-26',
         '2021-10-08',
         '2021-10-20',
         '2021-11-01',
         '2021-11-13',
         '2021-11-25',
         '2021-12-07',
         '2021-12-19',
         '2021-12-31',
         '2022-01-12',
         '2022-01-24',
         '2022-02-05',
         '2022-03-01',
         '2022-03-13',
         '2022-03-25',
         '2022-04-06',
         '2022-04-18',
         '2022-04-30'       ]

In [ ]:
area = [1.4059533823468258, 
        1.3092648591408815,
        4.089814860824349,
        35.45327379977961,
        164.35816819309844,
        189.77740718269564,
        196.40853083558454,
        202.39123215378402,
        203.523554334184,
        189.53771996241093,
        189.28488791741248,
        176.95780015366395,
        174.61886310791974,
        168.85974566428175,
        165.66604838008865,
        165.7250458032984,
        165.59976974259482,
        164.8396803411305,
        165.99755672579104,
        93.8793604665423,
        165.56881891971852,
        165.07898539829688,
        165.03871482093427,
        164.42640445435802,
        165.27437342244127,
        164.4474745820135,
        146.68979948818443,
        79.88630437641581,
        146.96696500476466,
        160.6680030299982,
        148.52494981096328,
        163.15544720448597,
        176.2540502817411,
        235.07805473025874,
        328.2007999738251,
        344.70344417990566,
        338.8554248933449,
        340.21834769774614,
        347.70020666359324,
        336.8797750371678,
        283.20250166097503,
        323.7673571733698,
        314.7612508283281,
        310.48020940622155,
        308.51955850905426,
        307.06320139583335,
        304.9742423852767,
        303.3029520921586,
        307.163230726144,
        301.9979937442847,
        290.2551696543628,
        302.44173942452807,
        296.0578490896608,
        271.2531198602065,
        258.9104369435108,
        252.94807243571785,
        236.89518025369952 ]


In [ ]:
water_level = [538,
               542,
               527,
               541,
               579,
               579,
               579,
               587,
               583,
               579,
               579,
               579,
               579,
               579,
               579,
               579,
               581,
               579,
               579,
               579,
               579,
               579,
               579,
               579,
               579,
               579,
               579,
               568,
               579,
               579,
               579,
               579,
               579,
               581,
               594,
               594,
               594,
               613,
               594,
               594,
               590,
               589,
               593,
               593,
               613,
               594,
               594,
               589,
               593,
               593,
               594,
               593,
               587,
               590,
               594,
               590,
               593,]


In [ ]:
volume = [0.0123237,
          0.01539,
          0.0709875,
          0.6283809,
          5.9219208,
          6.4411542,
          6.53715,
          8.2414476,
          7.4768265,
          6.3805725,
          6.4235412,
          6.2126901,
          6.1687656,
          6.035148,
          5.9746986,
          6.0095394,
          6.3389061,
          5.9972868,
          6.0131547,
          3.0654513,
          6.0114618,
          5.9967477,
          6.0017409,
          5.9911092,
          5.9998428,
          5.976495,
          5.5968498,
          1.9974285,
          5.2671636,
          5.9109993,
          5.5911636,
          5.9575527,
          6.1664454,
          7.5543867,
          12.6935667,
          12.9778416,
          12.852666,
          19.35774,
          13.0492584,
          12.8430873,
          10.170756,
          10.954836,
          12.1226508,
          12.0646881,
          18.1796814,
          12.3077745,
          12.2312277,
          10.6249797,
          12.018474,
          11.7944388,
          11.4739002,
          11.9239137,
          9.9836145,
          10.3547115,
          11.2307391,
          10.0941012,
          10.0561617]

In [ ]:
{'2021-01-29': 1611890718000}

In [ ]:
{'2021-05-05': 1620185119000}

In [ ]:
{'2021-09-02': 1630553126000}

In [ ]:
{'2021-11-25': 1637810726000}